# Pokemon_Computer_Vision_Project

Coded by Luna McBride. This dataset only comes with types, but I intend to add generation number to the mix, in an attempt to see if the various claims about how obviously new, and thus "bad", the newer ones are.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import tensorflow as tf #Import tensorflow in order to use Keras
from keras import backend as K #Used for trying to clear memory, but I could still not clear enough RAM

from tensorflow.keras.preprocessing.sequence import pad_sequences #Add padding to help the Keras Sequencing
import tensorflow.keras.layers as L #Import the layers as L for quicker typing
from tensorflow.keras.optimizers import Adam #Pull the adam optimizer for usage

from tensorflow.keras.losses import SparseCategoricalCrossentropy #Loss function being used
from sklearn.model_selection import train_test_split #Train Test Split
from tensorflow.keras.preprocessing import image #Add image handling, because I am looking at images
from PIL import Image #Pillow images, as that is the format Keras uses

from keras.models import Sequential #Sequential
from keras.layers import Conv2D, MaxPooling2D #Load 2d layers
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization #Load important layers

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pokemon-images-and-types/pokemon.csv
/kaggle/input/pokemon-images-and-types/images/images/luxio.png
/kaggle/input/pokemon-images-and-types/images/images/oshawott.png
/kaggle/input/pokemon-images-and-types/images/images/parasect.png
/kaggle/input/pokemon-images-and-types/images/images/taillow.png
/kaggle/input/pokemon-images-and-types/images/images/porygon.png
/kaggle/input/pokemon-images-and-types/images/images/feraligatr.png
/kaggle/input/pokemon-images-and-types/images/images/eelektross.png
/kaggle/input/pokemon-images-and-types/images/images/scyther.png
/kaggle/input/pokemon-images-and-types/images/images/magmar.png
/kaggle/input/pokemon-images-and-types/images/images/swalot.png
/kaggle/input/pokemon-images-and-types/images/images/magneton.png
/kaggle/input/pokemon-images-and-types/images/images/skitty.png
/kaggle/input/pokemon-images-and-types/images/images/stunfisk.png
/kaggle/input/pokemon-images-and-types/images/images/clawitzer.png
/kaggle/input/pokemon-images-and

In [2]:
pokemon = pd.read_csv("../input/pokemon-images-and-types/pokemon.csv") #Load the Pokemon
pokemon.head() #Take a peek at the pokemon

,Name,Type1,Type2
0,bulbasaur,Grass,Poison
1,ivysaur,Grass,Poison
2,venusaur,Grass,Poison
3,charmander,Fire,NaN
4,charmeleon,Fire,NaN


---

# Check for Nulls

In [3]:
print(pokemon.isnull().any()) #Print if there are any nulls

Name     False
Type1    False
Type2     True
dtype: bool


It looks like the issue is the dataset has nulls for pokemon that are not dual type. I think I will go with the gen 1 strategy and have the second type as the same as the first. For example, Charmander was considered a fire fire type in the code of Pokemon Red.

In [4]:
#DualTypeMaker: Makes a pokemon a dual type of the same type if their second type is null
#Input: The two types it is
#Output: the type type2 will become
def dualTypeMaker(type1, type2):
    if pd.isnull(type2): #If the second type is null
        return type1 #Make the second type the same as the first
    return type2 #Leave the already dual type alone

pokemon["Type2"] = pokemon.apply(lambda x: dualTypeMaker(x["Type1"], x["Type2"]), axis = 1) #Set the second type of the pokemon
pokemon.head() #Take a peek at the dataset

,Name,Type1,Type2
0,bulbasaur,Grass,Poison
1,ivysaur,Grass,Poison
2,venusaur,Grass,Poison
3,charmander,Fire,Fire
4,charmeleon,Fire,Fire


---

# Add a row for the Pokemon generation

The generation is essentially the set of games they were introduced. I happen to know this already, but I will type them here for those who do not know:

Gen 1: Pokemon 1-151 (Red and Green/Blue)

Gen 2: Pokemon 152-251 (Gold and Silver)

Gen 3: Pokemon 252-386 (Ruby and Sapphire)

Gen 4: Pokemon 387-493 (Diamond and Pearl)

Gen 5: Pokemon 494-649 (Black and White)

Gen 6: Pokemon 650-721 (X and Y)

Gen 7: Pokemon 722-809 (Sun and Moon)

This Dataset only goes through generation 7.

In [5]:
gen = [] #A list to hold the generation numbers
length = len(pokemon) #Get the number of pokemon in the dataset

#A for loop to put the generation into the list, going based on the list above -1 because the index starts at 0
for i in range(0, length):
    if i<=150:
        gen.append(1)
    elif i<=250:
        gen.append(2)
    elif i<=385:
        gen.append(3)
    elif i<=492:
        gen.append(4)
    elif i<=648:
        gen.append(5)
    elif i<=720:
        gen.append(6)
    else:
        gen.append(7)
        
#Get the pokemon that boarder each generation so we can make sure there were no mistakes
boarderPokemon = ["mew", "chikorita", "celebi", "treecko", "deoxys-normal", "turtwig", "arceus", "victini", "genesect", "chespin",
                 "volcanion", "rowlet", "melmetal"]
        
pokemon["Gen"] = gen #Insert the generation numbers into the dataset

pokemon["Gen"] = pokemon["Gen"].astype(str) #Change the gen to strings

print("Name, Generation\n") #Print the format of the pokemon prints

#For each pokemon, find the boarder pokemon and print their name and generation number
for i in range(0, length):
    if pokemon["Name"][i] in boarderPokemon: #If the pokemon is a boarder pokemon
        print(pokemon["Name"][i], ", ", pokemon["Gen"][i]) #Print the name and generation of that pokemon

pokemon.head() #Take a peek at the dataset

Name, Generation

mew ,  1
chikorita ,  2
celebi ,  2
treecko ,  3
deoxys-normal ,  3
turtwig ,  4
arceus ,  4
victini ,  5
genesect ,  5
chespin ,  6
volcanion ,  6
rowlet ,  7
melmetal ,  7


,Name,Type1,Type2,Gen
0,bulbasaur,Grass,Poison,1
1,ivysaur,Grass,Poison,1
2,venusaur,Grass,Poison,1
3,charmander,Fire,Fire,1
4,charmeleon,Fire,Fire,1


Judging by the boarder pokemon, all generation numbers were assigned correctly. Forme names are still a problem, but I will fix that later, because the image names have the forme too

---

# Get the Images

Image Processing: https://keras.io/api/preprocessing/image/

In [6]:
pokemonImages = [] #A list to hold the image data, which will then be put into the dataframe
datagen = image.ImageDataGenerator()

#For each pokemon, get and convert their image
for i in range(0, length):
    pokemonName = pokemon["Name"][i] #Get the pokemon name
    
    try: #Try to load the image assuming it is a png
        path = "../input/pokemon-images-and-types/images/images/{}.png".format(pokemonName) #Get the image based on the pokemon
        img = image.load_img(path, target_size = None, interpolation = "nearest") #Load the image of the pokemon
    except: #Catch the issue if the image is not a png, try a jpg
        path = "../input/pokemon-images-and-types/images/images/{}.jpg".format(pokemonName) #Get the image based on the pokemon
        img = image.load_img(path, target_size = None, interpolation = "nearest") #Load the image of the pokemon
    
    pokemonImages.append(path) #Append the image to the image list
    
pokemon["Image"] = pokemonImages #Put the image list into a new column "Image"
pokemon.head() #Take a peek at the dataset

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


,Name,Type1,Type2,Gen,Image
0,bulbasaur,Grass,Poison,1,../input/pokemon-images-and-types/images/image...
1,ivysaur,Grass,Poison,1,../input/pokemon-images-and-types/images/image...
2,venusaur,Grass,Poison,1,../input/pokemon-images-and-types/images/image...
3,charmander,Fire,Fire,1,../input/pokemon-images-and-types/images/image...
4,charmeleon,Fire,Fire,1,../input/pokemon-images-and-types/images/image...


In [7]:
#Split the images and gens into train and test
imageTrain, imageTest, genTrain, genTest = train_test_split(pokemon["Image"], pokemon["Gen"], test_size = 0.33)

In [8]:
trainDf = pd.DataFrame(imageTrain) #Put the training images into a dataframe
trainDf["Gen"] = genTrain #Add the generation to the dataframe
train = datagen.flow_from_dataframe(trainDf, x_col = "Image", y_col = "Gen") #Make a flow variable for keras

Found 542 validated image filenames belonging to 7 classes.


In [9]:
testDf = pd.DataFrame(imageTest) #Put the testing images into a dataframe
testDf["Gen"] = genTest #Add the generation to the dataframe
test = datagen.flow_from_dataframe(testDf, x_col = "Image", y_col = "Gen") #Make a flow variable for keras

Found 267 validated image filenames belonging to 7 classes.


---

# Create the Model

Note, we are checking for generation labeling accuracy here

Sources: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html , https://keras.io/api/preprocessing/image/#imagedatagenerator-class

In [10]:
tf.keras.backend.clear_session() #Clear any previous model building

epoch = 2 #Number of runs through the data
batchSize = 8 #The number of items in each batch
width = 256 #The width of the images
height = 256 #The height of the images
channels = 3 #The number of channels (RGB)

model = Sequential() #Add a sequential to the model
model.add(L.Lambda(lambda x: x, input_shape = (width, height, channels))) #Put the input into a lambda, because it would not work for some reason in the Conv2D
model.add(Conv2D(32, (3, 3))) #Add a convolutional image layer
model.add(BatchNormalization()) #Normalize the data
model.add(Activation('relu')) #Make the activation relu to discourage negative units
model.add(MaxPooling2D(pool_size=(2, 2))) #Max pool the data to keep the most important characteristics

model.add(Conv2D(32, (3, 3))) #Add a convolutional image layer
model.add(Activation('relu')) #Make the activation relu to discourage negative units
model.add(MaxPooling2D(pool_size=(2, 2))) #Max pool the data to keep the most important characteristics

model.add(Conv2D(64, (3, 3))) #Add a bigger convolutional image layer, layering activations
model.add(Activation('relu')) #Make the activation relu to discourage negative units
model.add(MaxPooling2D(pool_size=(2, 2))) #Max pool the data to keep the most important characteristics

model.add(Flatten()) #Make the layers flat to apply the characteristics into one slot
model.add(Dense(64)) #Add a dense layer to track activation
model.add(Activation('relu')) #Make the activation relu to discourage negative units
model.add(Dropout(0.2)) #Have a 0.2 dropout to prevent overfitting
model.add(Dense(1)) #Add another dense layer to finish the lot
model.add(Activation('sigmoid')) #Make the activation sigmoid 

model.compile(loss = 'binary_crossentropy', #Make the loss binary to fit with the sigmoid endpoint
              optimizer = 'rmsprop', #Use root mean squared prop to optimize (adam came to similar results at much slower speeds)
              metrics = ['accuracy']) #Track the accuracy of the model

history = model.fit_generator(train, epochs = epoch) #Fit the model to the data

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch 1/2
17/17 [==============================] - 36s 2s/step - loss: 2.9541 - accuracy: 0.8321
Epoch 2/2
17/17 [==============================] - 30s 2s/step - loss: 0.4518 - accuracy: 0.8571


---

# Make Predictions

In [11]:
loss, accuracy = model.evaluate(test) #Get the loss and Accuracy based on the tests

#Print the loss and accuracy
print("Test Loss: ", loss)
print("Test Accuracy: ", accuracy)

9/9 [==============================] - 4s 420ms/step - loss: 0.4248 - accuracy: 0.8571
Test Loss:  0.42476698756217957
Test Accuracy:  0.8571428656578064


85.7% accuracy is pretty telling. That means that there is something to each generation of Pokemon that set the designs apart, rather than just an old new split. I wanted to delve deeper into actual predictions and classification reports, but ram went through the roof, even with a 0.01 test size. Nothing I could find online fixed it, so I am going to say that the fact that the model can predict the exact generation number with 85% accuracy tells that each generation has distinct designs that cannot be shoved into a new/old split.